In [1]:
# Imports
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from ResBlock import ResidualBlock

In [3]:
# Load database
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"Total training example: {x_train.shape[0]}, \nTotal test examples: {x_test.shape[0]}")

Total training example: 60000, 
Total test examples: 10000


In [3]:
# Create a folder for test images
image_test_path = "./images_test"
if not os.path.exists(image_test_path):
    os.mkdir(image_test_path)


def arraytoimage(x, y):
    """Converts numpy array to image"""
    len_x = x.shape[0]
    for i in range(len_x):
        image = Image.fromarray(x[i])
        name = str(y[i])
        image.save(f"{image_test_path }/{name}.png")

In [4]:
arraytoimage(x_test, y_test)

In [14]:
# Scale test and training set to range from 0 to 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [16]:
# Building a ResNet-12 model

model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28,1)))
# Keep the first stack of layers of ResNet with a little change in the
# number of strides, set to 1 instead of 2 (due to the size of images)
model.add(keras.layers.Conv2D(64, kernel_size=7, strides=1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=1, padding="same"))

# Number of filters in the first RB
pre_filter = 64

# Build the residual blocks using ResidualBlock class
# defined in the ./ResBlock/residual.py file
for filter in [64] * 3 + [128]*2:
    strides = 1 if filter == pre_filter else 2
    model.add(ResidualBlock(filters=filter, strides=strides))
    pre_filter = filter

# Uses a glabal average layer and then flattens its output
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())

# Add the output layer
model.add(keras.layers.Dense(10, activation="softmax"))

In [18]:
# Adam optimizer and compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 22, 22, 64)          │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 22, 22, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_5 (ResidualBlock)     │ (None, 22, 22, 64)          │          74,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_6 (ResidualBlock)     │ (None, 22, 22, 64)          │          74,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_7 (ResidualBlock)     │ (None, 22, 22, 64)          │          74,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_8 (ResidualBlock)     │ (None, 11, 11, 128)         │         230,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_9 (ResidualBlock)     │ (None, 11, 11, 128)         │         295,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 754,314 (2.88 MB)

 Trainable params: 752,138 (2.87 MB)

 Non-trainable params: 2,176 (8.50 KB)

In [20]:
# Train the model
model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=5,
    validation_split=0.2
)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 502s 335ms/step - accuracy: 0.9849 - loss: 0.0463 - val_accuracy: 0.9682 - val_loss: 0.1068
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 498s 332ms/step - accuracy: 0.9883 - loss: 0.0367 - val_accuracy: 0.9882 - val_loss: 0.0439
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 514s 343ms/step - accuracy: 0.9900 - loss: 0.0322 - val_accuracy: 0.9730 - val_loss: 0.0912
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 491s 327ms/step - accuracy: 0.9911 - loss: 0.0300 - val_accuracy: 0.9761 - val_loss: 0.0886
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 476s 317ms/step - accuracy: 0.9921 - loss: 0.0259 - val_accuracy: 0.9878 - val_loss: 0.0403


In [21]:
# Evaluate the model
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 101ms/step - accuracy: 0.9864 - loss: 0.0477


[0.037882786244153976, 0.9890000224113464]

In [24]:
# Save the model
model.save("./app/models/model.keras")

In [49]:
# Test
model = keras.models.load_model("./app/models/model.keras")
np.argmax(model.predict(np.expand_dims(x_test[0],0)))==y_test[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step


True

In [5]:
file_bytes = b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00'

array = np.asarray(bytearray(file_bytes), dtype=np.uint8)

print(array)

[255 216 255 224   0  16  74  70  73  70   0   1   1   0]
